# Contents
## 1. Data collection and storage
- Crawler script key code
- How to use

## 2. Data preparation
- Import Module
- Load the data and fetch the data set
- word segmentation

## 3. Exploration and visualization
- The distribution of views
- The words in the training set

## 4. Experimentation and prediction
- Neural Networks: CNN (train and prediction)
- Machine learning: Bayes

# Data collection and storage

- key code
- you can run this cell but it will cost a lot of time, and you can also run spider.py directly

In [ ]:
def run_spider():
    import requests
    from lxml import etree
    import re
    import time #有必要时用来睡眠
    import pandas as pd 
    # version1：存在两个问题1是link有时候不是cqu而微信等来源2是最后一次再写文件不安全（每隔几轮就保存）
    # version2：也有两个问题1是速度太慢可以用多进程和异步io解决2是读写文件每次重写csv可以改成追加且list变frame再保存太麻烦直接写或者csv库
    # version3：加入了多进程没;对比了csv模块pd更快尤其存pkl更快；修复了nexturl会死循环；修复浏览数为零bug
    # 使用追加而非内存保存全部的数据；但避免中断运行后追加重复命名加上随机
    start = time.time()
    url="http://news.cqu.edu.cn/newsv2/news-126.html"
    url2 = "http://news.cqu.edu.cn/newsv2/news-126.html?&page=590" #测试url
    url3 = "https://news.cqu.edu.cn/newsv2/news-132.html"

    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}
    results=[]
    flag = True
    while flag:
        html = requests.get(url, headers=header).text
        # print(html)
        listpage = etree.HTML(html)
        # lis = listpage.xpath('/html/body/div[3]/div/div[2]') 
        lists = listpage.xpath('//div[@class="lists"]/div[@class="item"]') #这里需要注意选择的是item

        # 一个page内的抓取
        for oneSelector in lists: # 收集的细节
            # testSelector = oneSelector.xpath("@class")[0]
            # print(testSelector) #item

            # 不同板块的div顺序还是不同要选class
            link = oneSelector.xpath("div[@class='content']/div[@class='title']/a/@href")[0]
            if "http://news.cqu.edu.cn" not in link:
                link = "http://news.cqu.edu.cn" + link
            title = oneSelector.xpath("div[@class='content']/div[@class='title']/a/text()")[0]
            abstract = oneSelector.xpath("div[@class='content']/div[@class='abstract']/text()")[0]
            try:
                html_sub = requests.get(link, headers=header).text
                newspage = etree.HTML(html_sub)
                datetime = (newspage.xpath("//div[@class='ibox']/span[2]/text()"))[0]
                author = newspage.xpath("//div[@class='dinfoa']/p/a/text()") #a个数不定的但可以这样加入到list
                # TODO: 这个count不能直接抓得找api发现在html里有这个api
                view_num = newspage.xpath("//span[@id='hits']/text()")[0]

                api = newspage.xpath('//script[@language="JavaScript"]/@src')[0] #$('#todaydowns').html('227');$('#weekdowns').html('227');$('#monthdowns').html('227');$('#hits').html('227');
                apiout = (requests.get(api,headers=header).text) 
                view_num = apiout.split("#hits').html('")[-1].replace("');", "") #更好的对上面字符串提取是正则
                print(view_num)
                tags = newspage.xpath("//div[@class='tags']/a/text()")
                # 有些正文的p不一样还有含img的 newsbody = newspage.xpath("//p[@style='text-align: justify;']/text()")
                newsbody = newspage.xpath("//div[@class='acontent']/p/text()")
                results.append([datetime,title,author,abstract,link,tags,newsbody,view_num])

            except:
                print("Error:",link)
        next_url = listpage.xpath('//div[@class="page"]/a[last()]/@href')[0]
        cur_page = listpage.xpath('//div[@class="page"]/span/text()')[0]

        if next_url and cur_page==next_url.split("=")[-1]:
            # url = "http://news.cqu.edu.cn/newsv2/"+ next_url
            print(cur_page,next_url.split("=")[-1])
            flag = False
        url = next_url
        print("crawl:"+url)
    filename = "data_set"+str(time.time())+".pkl"
    df = pd.DataFrame(results,columns=['datetime','title','author','abstract','link','tags','newsbody','view_num']) #,columns=['link','title','abstract']
    # df.to_csv(filename, mode='a',encoding='utf_8_sig')
    df.to_pickle(filename)
# run_spider()

# Data preparation
- The data is saved in the PKL format because it is fast to read and write and is small in volume.I have also saved a copy of the CSV for easy viewing, so we can load the file directly here.

In [ ]:
# import
import os
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore') 
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

- load data and stopwords

In [ ]:
def get_stopwords(stopwords_dir="../input/cqunews/stopwords"):
    """
       读取停用词库,这里只是用了一个停用词库   如果要预测结果更高点的话，可以将多个语料库结合起来使用
       :return: set 包含的停用词
       """
    files = os.listdir(stopwords_dir)#返回路径下的所有文件
    stopwords = []
    for f in files:
        filepath = os.path.join(stopwords_dir, f)#把文件拼接成文件路径
        with open(filepath, 'r', encoding='utf-8') as r:
            w = r.read().split()
        stopwords += w
    return set(stopwords) #b把所有停用词文件里的都取出来，然后放到集合里

In [ ]:
filepath = [r"/kaggle/input/cqunews/zhaosheng.pkl",\
            r"/kaggle/input/cqunews/zonghe.pkl"]
stopwords = get_stopwords("/kaggle/input/cqunews/stopwords")
data = (pd.concat([pd.read_pickle(file) for file in filepath]))
X = data.drop(['view_num'],axis=1)
y = data['view_num'].astype(int) #转换数字
data.head() #一万八条 

- View the pageview distribution

In [ ]:
# max(y) 54514
# min(y) 9
# (y.mean()) 798
# y.median() 489 中位数
# y.plot() 这个图x是序号没什么意义
# ct = Counter(y)
# plt.bar(list(ct.keys()),list(ct.values()),label='xl1')#label为设置图例标签，需要配合legend（）函数才能显示出
# (sorted(ct.items(), key=lambda data: data[1])) 不是直方图也不是折线图
# y.plot(kind='bar') 太慢了
plt.hist(y,range=(0,y.mean()*2),bins=10) # 原本氛围是0到最大看到基本分布在一侧于是确定范围

 Processing the label
<!-- - 离散问题/二分类/多分类
- 保留离散数据当作回归预测浏览数
- 标签二值化（标准？均值/中位数）
- 标签区间化（多少个？按照最小最大均分？指数型区间？均值为中心分？） -->

In [ ]:
label = y.apply(lambda x:x>y.mean()) # 看图就知道用均值为负较多
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
ystd = standardscaler.fit_transform(np.array(y).reshape(-1,1))
Counter(label)

- Use Jieba for Chinese word segmentation and take out the X (We chose to focus only on the abstract column, and we also tried to merge the tags columns without much effect)
>特征选择  取一列？取多列合并一列？取多列？

In [ ]:
# 使用title+tags合并，且后者不需要分词
import jieba.posseg as pseg
def content_to_word(line):
    gen = pseg.cut(line)
    words = []
    for i in gen:
        if i.flag != 'x' and i.word not in stopwords  and not i.word.isdigit(): #停用词
            words.append(i.word) #只加非标点符号
    return ' '.join(words) #空格分割的字符串
# data['tags']+data['title'] 需要先tags处理
# temp = data['tags'].map(lambda x: " ".join(x)) #多运行几次会被分词单字因为tags会反复使用所有改用temp存数据
# data['title'] = data['title'].map(content_to_word)
# X = data['title'] #+ temp
X = data['abstract'].map(content_to_word)
X

# Exploration and visualization
- This section is going to visualize X and manipulate X to make it a better representation

In [ ]:
import wordcloud
def word_cloud_analyse(datas,stopwords):
    X=datas['abstract']
    words=[]
    for i in range(len(X)):
        l=[]
        l=jieba.lcut(X.iloc[i])
        l=[t for t in l  if t not in stopwords and t!=' ']
        words+=l
    words=' '.join(words)
#     ,font_path="C:\\Windows\\Fonts\\HGWYS_CNKI.TTF"
    wd=wordcloud.WordCloud(background_color="white")
    wd.generate(words)
    return wd
        

stopwords=get_stopwords()
wd=word_cloud_analyse(data,stopwords)
%pylab inline
import matplotlib.pyplot as plt
plt.imshow(wd, interpolation='bilinear')
plt.axis("off")

# Experimentation and prediction
- This section tries neural network CNN and machine learning Bayes and makes predictions

In [ ]:
# This function returns the test set training set after word vectorization
def tokenizer_data(X, y, MAX_SEQUENCE_LENGTH):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    joblib.dump(tokenizer, './models/Tokenizer')

    sequences = tokenizer.texts_to_sequences(X)
    word_index = tokenizer.word_index
    print('数据集含 %s 个不同的词.' % len(word_index))
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    labels = to_categorical(np.asarray(y)) # onehot 
    labels = np.asarray(y)
    print('Shape of data tensor:', data.shape,"[0]:", data[0])
    print('Shape of label tensor:', labels.shape,"[0]:", labels[0])


    # 第一部分划分应该放这儿/模型fit报错就是因为直接用的原始数据
    X, X_test, y, y_test = train_test_split(data, labels, test_size=.20, random_state=2048)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.10, random_state=2048)
    return word_index,X_train,y_train,X_val,y_val,X_test,y_test
word_index, X_train, y_train, X_val, y_val, X_test, y_test = tokenizer_data(X,label,MAX_SEQUENCE_LENGTH) #用ystd是做回归的

- This cell is about training CNN. If there is no GPU, it may be slow, so you can skip this part and use the trained files directly

In [ ]:
import joblib
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
# train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)
from tensorflow.keras import regularizers
from tensorflow.keras import initializers

MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 200
def train():
    print('3. #############载入word2vec...##########')
    embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)
    print('4. #############模型及训练...#############')

    model = Sequential()
    # 使用上面构造的layer
    model.add(embedding_layer)
    model.add(Dropout(0.3))
    model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1)) 
    model.add(MaxPooling1D(3))
#     model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(EMBEDDING_DIM, activation='relu'))
#     model.add(Dense(EMBEDDING_DIM, activation='relu',kernel_regularizer=regularizers.l2(0.01)))

    # model.add(Dense(y_train.shape[1], activation='softmax')) #多分类softmax
    model.add(Dense(1, activation='sigmoid')) #二分类
#     model.add(Dense(1)) #回归

    model.summary()  
    from tensorflow.python.keras.utils.vis_utils import plot_model
    plot_model(model, to_file='cnn_model.png',show_shapes=True)
    cce = tf.keras.losses.CategoricalCrossentropy()
    bce = tf.keras.losses.BinaryCrossentropy()
    # mse = tf.keras.losses.MSE()
    opt = tf.keras.optimizers.SGD(learning_rate=0.1)
    model.compile(loss=bce,
                  optimizer='rmsprop', #rmsprop
                  metrics=['acc']) #'mae', 'mse','acc'
    print(model.metrics_names)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=128) #loss为零
    model.save('./models/cnn_word.h5')


    print('4. #############评估...#############')
    print(model.evaluate(X_test, y_test))
    return model
model = train()

![](cnn_model.png)

#### A summary of the model
- We see that the loss of verification set increases first and then decreases in training, indicating that CNN overfits this model, which may be caused by label binarization and data volume
- 用错损失函数会导致训练不能进行
- 初始使用cnn会过拟合即loss下降val loss先降后升
- 将模型输出改为多分类（onehot编码如1变成0 1）改动没用
- 改成回归loss非常大
- 使用正则化项反而训练没有效果 [keras文档](https://keras-cn.readthedocs.io/en/latest/other/regularizers/)
- 优化器方面：SGD是优化不了，adam虽然更快但也是先下降后升
- 想到不管标签如何划分二类只是影响acc而loss变化是一样的所以划分没多大意义，另一方面变成回归只有loss指标且也是先降低再震荡
- 换成abstract列代替title会好些另外用新闻主题代替呢

- Here's how to make predictions

In [ ]:
def word_to_vector(text, MAX_SEQUENCE_LENGTH=100):
    tokenizer = joblib.load("./models/Tokenizer")
    word_index = tokenizer.word_index
    sequences = tokenizer.texts_to_sequences([text])
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
#     data = pad_sequences(sequences)

    return data
def predict(sentence):
    model = tf.keras.models.load_model('./models/cnn_word.h5')

    gen = pseg.cut(sentence)
    words = []
    for i in gen:
        if i.flag != 'x' and i.word not in stopwords and not i.word.isdigit() :
            words.append(i.word)  
    data = ' '.join(words) 
    data = word_to_vector(data)
    print(((model.predict(data)))) #直接model调用
predict("为庆代，2021年5月，学校举办了“中国梦•劳动’美个学院踊跃参加。")
predict("6月3日下午，联谊赛决赛在重大花园乒乓球馆拉开战幕。从学院教职工和硕士研究生选拔出的16名选手参加了决赛")
# Since we take the average above as a positive sample, we need the page views greater than 800 to be greater than 0.5

In [ ]:
data['abstract'].iloc[0]

In [ ]:
# Attempt decision tree 
# from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,AdaBoostClassifier #,DecisionTreeClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.feature_selection import RFE
# from sklearn.decomposition import PCA, TruncatedSVD, PCA
# from mlxtend.classifier import EnsembleVoteClassifier 
# DTC_Classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
# ETC_Classifier = ExtraTreesClassifier()
# RTC_Classifier = RandomForestClassifier(criterion='entropy')
# #  Gaussian Naive Baye Model
# BNB_Classifier = GaussianNB()

# # GBM
# import xgboost as xgb
# XGB_Classifier = xgb.XGBClassifier()
# # ADA
# ADA_Classifier = AdaBoostClassifier(
#     DTC_Classifier,
#     n_estimators=100,
#     learning_rate=1.5)
# import lightgbm as lgb
# params = {
#     "objective": 'binary',
#     "num_leaves": 24,
#     "learning_rate": 0.2,
#     "bagging_fraction": 0.9440403047411987,
#     "bagging_freq": 8
# }
# LGB_Classifier = lgb.LGBMClassifier(objective='binary',num_leaves=31,learning_rate=0.2,bagging_fraction=0.8963,bagging_freq=4)

In [ ]:
# # Evaluate Models
# from sklearn.model_selection import cross_val_score
# from sklearn import metrics

# models = []
# # models.append(('Decision Tree Classifier', DTC_Classifier))  #简直过拟合了训练集上近1的acc且绝大多数的预测概率直接1
# # models.append(('Random Forest Classifier', RTC_Classifier))
# # models.append(('Extract Tree Classifier', ETC_Classifier))
# models.append(('BNB Classifier', BNB_Classifier))


# # models.append(('XGB Classifier', XGB_Classifier)) #xgb的fit耗时很长但效果比lgb好点，经测predict高点
# # models.append(('LGB Classifier', LGB_Classifier))
# # models.append(('LogisticRegression', LGR_Classifier))
# def train_evaluate(x,y,istest=False):
#     for i, v in models:
#         if istest==False:
#             v.fit(x,y)
#         t1 = time.time()
#         vpred = v.predict(x)
# #         scores = cross_val_score(v, x, y, cv=10) #单就这一句就用了cv次fit函数太费时间
#         accuracy = metrics.accuracy_score(y, vpred) #和model.score一样作用
#         cm = metrics.confusion_matrix(y, vpred)
#         classification = metrics.classification_report(y, vpred)
#         print('Cost',time.time()-t1)
#         print('============================== {} Model Evaluation =============================='.format(i))
#         print()
# #         print ("Cross Validation Mean Score:" "\n", scores.mean())
#         print ("Model Accuracy:" "\n", accuracy)
#         print(f"TNR:{cm[1][1]/(cm[0][1]+cm[1][1])}\nTPR:{cm[0][0]/(cm[0][0]+cm[1][0])}" "\n")
#         print("Classification report:", classification)
# #         show_feature_importance(v)
#         fpr,tpr,thr = metrics.roc_curve(y,v.predict_proba(x)[:,1],1)
# #         display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=metrics.auc(fpr,tpr),estimator_name=i)
#         ac = metrics.auc(fpr,tpr)
#         plt.plot(fpr,tpr,label=f"{i}:AUC={ac:1.5f}") #scatter画点可见是离散
#         print('FPR:\t',fpr.shape)#是一每个
# #         metrics.plot_roc_curve(v,x,y) 为了画在一个图里用RocCurveDisplay(也不能) 但这个是散点显示折线（默认的thr取值是按照score而不是0-1之间依次取）
#         plt.legend() # 设置位置，没有这个label不生效(还必须在每此plot后)
#     plt.ylabel('TPR(真正类率：实际正中预测正)',fontsize='15')
#     plt.xlabel('FPR(假正类率：实际负中预测正)',fontsize='15')
#     plt.title('ROC(接收者操作特征，每个阈值确定个点)',fontsize='20')
#     plt.show()
#     plt.savefig('roc.png')
#     from IPython.display import FileLink
#     FileLink('roc.png')
# import time
# word_index, X_train, y_train, X_val, y_val, X_test, y_test = tokenizer_data(X,y,MAX_SEQUENCE_LENGTH=80) #用y是做回归的
# X_test,y_test
# Counter(label)
# # train_evaluate(X_train,y_train) 
# # train_evaluate(X_test,y_test,True) 